In [ ]:
import tqdm
import numpy as np
import pandas as pd
from datetime import datetime
import sys

sys.path.append('../../')
from utils import file_utils, microarray_utils, clustering_utils

In [ ]:
partition_type = 'EXP'
full_filtered_df = pd.read_csv('../microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv')
full_filtered_df = full_filtered_df.rename(columns={'Unnamed: 0': 'TTHERM_ID'})

In [ ]:
full_filtered_df.head()

In [ ]:
# full_filtered_df = shuffle_rows(full_filtered_df)
# partition_type = 'NC'

In [ ]:
full_filtered_df.head()

In [ ]:
full_filtered_norm_df = microarray_utils.normalize_expression_per_gene(full_filtered_df)
raw_data = full_filtered_norm_df[list(full_filtered_norm_df.columns)[1:]].values

# SCAN START

In [ ]:
curr_datetime = str(datetime.now())

In [ ]:
idx_labels = list(range(raw_data.shape[0]))

p_minkowski = None
metrics = ['clr', 'manhattan', 'euclidean', 'cosine'] + [f'minkowski_{str(p)}' for p in np.array([0.5, 1, 2, 3, 4, 5])]
# metrics = ['manhattan']
n_jobs = -1
random_state = 42

In [ ]:
scan_nns = np.arange(2, 13, 1)
# scan_nns = [5]
scan_nns

In [ ]:
# scan_rps = np.arange(0.005, 1.1, 0.005)
scan_rps = np.arange(0.1, 1.1, 0.1)
# scan_rps = [0.030, 0.035]
scan_rps

In [ ]:
scan_dict = {}

In [ ]:
for metric_p in metrics:
    metric_p_split = metric_p.split('_')

    metric = metric_p

    if metric_p_split[0] == 'minkowski':
        metric = metric_p_split[0]
        p_minkowski = float(metric_p_split[1])

    if metric != 'clr':
        distance_matrix = clustering_utils.compute_pairwise_distance_matrix(raw_data, metric, n_jobs, p_minkowski)
    else:
        distance_matrix = clr_dist_arr

    nn_idxs, nn_dists = clustering_utils.compute_nns(raw_data, max(scan_nns), metric, random_state, n_jobs, p_minkowski, distance_matrix)

    for idx, nn in enumerate(scan_nns):     
        print(idx+1,'of',len(scan_nns))     
        print('NNs: ', nn)

        scan_dict[nn] = {}

        scan_dict[nn]['nn_idxs'] = nn_idxs
        scan_dict[nn]['nn_dists'] = nn_dists

        nn_graph = clustering_utils.compute_umap_graph(raw_data, nn, metric, nn_idxs, nn_dists)
        scan_dict[nn]['nn_graph'] = nn_graph

        for rp in tqdm.tqdm(scan_rps):

            scan_dict[nn][rp] = {}
            
            partition = clustering_utils.compute_leiden_partition(nn_graph, rp, random_state)
            scan_dict[nn][rp]['partition'] = partition

            communities = clustering_utils.compute_communities(partition, idx_labels)
            scan_dict[nn][rp]['communities'] = communities

            sil_score = clustering_utils.compute_silhouette_score(distance_matrix, partition)
            scan_dict[nn][rp]['sil_score'] = sil_score

            modularity = clustering_utils.compute_modularity(nn_graph, communities.values())
            scan_dict[nn][rp]['modularity'] = modularity

            enrichment_df = clustering_utils.compute_enrichment(full_filtered_norm_df, partition)
            scan_dict[nn][rp]['enrichment_df'] = enrichment_df

            num_clusters = clustering_utils.compute_num_clusters(partition, communities.values())
            scan_dict[nn][rp]['num_clusters'] = num_clusters

            num_enriched_clusters = clustering_utils.compute_num_enriched_clusters(enrichment_df)
            scan_dict[nn][rp]['num_enriched_clusters'] = num_enriched_clusters

            num_enriched_cluster_genes = clustering_utils.compute_num_enriched_cluster_genes(enrichment_df, partition)
            scan_dict[nn][rp]['num_enriched_cluster_genes'] = num_enriched_cluster_genes

            cluster_sizes = clustering_utils.compute_cluster_sizes(communities)
            scan_dict[nn][rp]['cluster_sizes'] = cluster_sizes

            enriched_cluster_sizes = clustering_utils.compute_enriched_cluster_sizes(communities, enrichment_df)
            scan_dict[nn][rp]['enriched_cluster_sizes'] = enriched_cluster_sizes

            cluster_stats = {
            'partition_type': partition_type,

            'dimensionality': 'baseline',

            'metric': metric_p,
            'graph': 'umap_fuzzy_simplicial_set',
            'nns': nn,

            'clustering': 'leiden_cpm',
            'parameter': rp,

            'silhouette_score': sil_score,
            'modularity': modularity,

            'nclusters': num_clusters,
            'mean_cluster_size': clustering_utils.compute_cluster_size_mean(cluster_sizes),
            'median_cluster_size': clustering_utils.compute_cluster_size_median(cluster_sizes),
            'sd_cluster_size': clustering_utils.compute_cluster_size_sd(cluster_sizes),

            'nenriched_clusters': num_enriched_clusters,
            'mean_enriched_cluster_size': clustering_utils.compute_cluster_size_mean(enriched_cluster_sizes),
            'median_enriched_cluster_size': clustering_utils.compute_cluster_size_median(enriched_cluster_sizes),
            'sd_enriched_cluster_size': clustering_utils.compute_cluster_size_sd(enriched_cluster_sizes),
            'nenriched_cluster_genes': num_enriched_cluster_genes,

            'datetime': curr_datetime
            }

            file_utils.write_to_csv('./scan_stats_mar2024.csv', cluster_stats, list(cluster_stats.keys()))

In [ ]:
pd.read_csv('./scan_stats_mar2024.csv')